In [1]:
import os 
import numpy as np
from hsflfm.util import MetadataManager, load_dictionary, save_dictionary
from hsflfm.processing import StrikeProcessor
from hsflfm.analysis import ResultPlotter, ResultManager, BulkAnalyzer
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
def get_filename(specimen_number, strike_number, alignment_folder):
    if not os.path.exists(alignment_folder):
        os.mkdir(alignment_folder)
    spec_folder = alignment_folder + f"/{specimen_number}"
    if not os.path.exists(spec_folder):
        os.mkdir(spec_folder)
    filename = spec_folder + f"/strike_{int(strike_number)}.json"
    return filename

def get_save_filename(specimen_number, strike_number, save_folder):
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    spec_folder = save_folder + f"/{specimen_number}"
    if not os.path.exists(spec_folder):
        os.mkdir(spec_folder)
    filename = spec_folder + f"/strike_{int(strike_number)}_results.json"
    return filename

save_folder = "../complete_results_pseudo_inverse_20250803"
alignment_folder = "../alignment_results_20241227"

In [ ]:
specimen_numbers = MetadataManager.all_specimen_numbers() 
for specimen_number in specimen_numbers:
    mm = MetadataManager(specimen_number)
    for strike_number in tqdm(mm.strike_numbers):
        save_filename = get_save_filename(specimen_number, strike_number, alignment_folder)
        if os.path.exists(save_filename):
            print(f"already completed {specimen_number}, {strike_number}")
            continue
        filename = get_filename(specimen_number, strike_number, save_folder)

        if not os.path.exists(filename):
            print(f"could not find file for {specimen_number} strike {strike_number}")
            continue

        align_results = load_dictionary(filename)

        processor = StrikeProcessor(align_results)
        processor.get_flow_vectors()
        processor.run_regression() 
        processor.get_relative_displacements() 
        result_info = processor.condense_info() 

        save_dictionary(result_info, save_filename)